In [9]:
from tensorflow import keras
import numpy as np
import nltk
import pickle
from keras.preprocessing.sequence import pad_sequences


# nltk.download("stopwords")
arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))

# loading models

In [2]:
seq2seq_model = keras.models.load_model('seq2seq_50epoch.h5')
encoder_model = keras.models.load_model('encoder_model.h5')
decoder_model = keras.models.load_model('decoder_model.h5')

# loading Tokenizers

In [3]:
with open('xtokenizer.pickle', 'rb') as handle:
    x_tokenizer = pickle.load(handle)
    
with open('ytokenizer.pickle', 'rb') as handle:
    y_tokenizer = pickle.load(handle)
    
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

# Constants

In [4]:
max_text_len=500
max_summary_len=13

# Preprocessing steps

In [5]:
import re
import string

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)



def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
       
def remove_stop_words(text):
    text = ' '.join([x for x in text.split(' ') if x not in arb_stopwords])
    return text

def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

def preprocessing(text):
    text = normalize_arabic(text)
    text = remove_diacritics(text)
    text = remove_punctuations(text)
    text = remove_repeating_char(text)
    text = remove_stop_words(text)
    text = clean_str(text)

    return text

In [6]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        
        
    
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token

        # why [0,-1,:] ??
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

# Encoding & Decoding text

In [7]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        
        
    
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token

        # why [0,-1,:] ??
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [8]:
def seq2summary(input_seq):
    # breakpoint()
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString






# Prepare text

In [1]:
text = ''' كد ياسر عثمان سفير مصر في المغرب، أن مباراة نادي الزمالك والوداد المغربي في دوري أبطال أفريقيا، ستقام بدون حضور جماهيري. '''

In [2]:
preprocessed = preprocessing(text)
test_seq  =   x_tokenizer.texts_to_sequences([preprocessed]) 
final_test_seq = pad_sequences(test_seq,  maxlen=max_text_len, padding='post')


NameError: name 'preprocessing' is not defined

In [22]:

final_test_seq= final_test_seq[0]
print("Review:",seq2text(final_test_seq))
# print("Original summary:",seq2summary(y_tr[i]))
print("Predicted summary:",decode_sequence(final_test_seq.reshape(1,max_text_len)))
print("\n")

Review: اكد وليد الركراكي مدرب نادي الوداد البيضاوي انه يعيش وقع الصدمه بسب هزيمه ناديه امام حسنيه اكادير مباراه مءجله الجوله 18 بالدوري المغربي واضاف الركراكي تصريحات تليفزيونيه انه يتوقع ظهور فريقه بهذا المستوي امام حسنيه اكادير وتابع انا اول يتحمل مسءوليه حدث الهزيمه الثانيه تواليا امر يحدث ان توليت القياده الفنيه لنادي وقد حدث الان لكنا نسمح الامر كونه ازمه كبيره واردف الركراكي نهنء حسنيه اكادير علي انتصاره لقد مستحقا انه موقعه الزمالك الي عديد الامور سنحاول العمل عليها واصل تنتظرنا مباراه مهمه امام الزمالك المجال لشك لينال منا امر مرغوب الان علينا طي الصفحه والتفكير المباراه المقبله في المقابل قال عبد الهادي السكتيوي مدرب حسنيه اكادير تصريح تلفزيوني لقناه الرياضيه ان ناديه عمل بشكل قوي ونال مكافاه المجهود بذله الاعبون التدريبات واضاف لعبنا امام فريق قوي الوداد يملك التاريخ والالقاب والانتصار بملعبه بالامر السهل وتابع السكتيوي لعبنا مباراه مثاليه قدمنا تدربنا وقام الاعبون علي ارض الواقع لذلك يستحقون مني التهنءه علي امل مواصله العمل 
Predicted summary:  start مدرب الوداد لا نخشي اي 